In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import csv
import numpy as np

DATA_DIR = 'MNIST'
sentiment_data = 'sentiment-data'


def getWordVectorDict():
    reader = csv.reader(open(sentiment_data +'/word-vectors-refine.txt'))

    word_vector_dict = {}
    for row in reader:
        key = row[0]
        if key in word_vector_dict:
            # implement your duplicate row handling here
            pass
        word_vector_dict[key] = np.array(row[1:])
    return word_vector_dict

def getPaddedSentenceMatrix(sentenceMatrix):
    wordCount = 100
    return np.vstack((sentenceMatrix, np.zeros((wordCount - np.shape(sentenceMatrix)[0],np.shape(sentenceMatrix)[1]), dtype=np.float32)))

def getVectorForSentence(sentence, word_vec_dict):
    sentence_matrix = []
    for word in sentence.split(' '):
        word_vec = word_vec_dict[word]
        if(len(sentence_matrix) == 0):
            sentence_matrix = word_vec
        else:
            sentence_matrix = np.vstack((sentence_matrix,word_vec))
    return getPaddedSentenceMatrix(sentence_matrix)

def getData(fileName):
    reader = csv.reader(open(sentiment_data +'/' + fileName))
    trainingData = []
    for row in reader:
        data = {}
        data['label'] =  1 if row[0] == 'postive' else 0
        data['sentence'] = row[1:]
        trainingData.append(data)
    return trainingData

word_vec_dict = getWordVectorDict()


def transform(row):
    return row['label'], getVectorForSentence(row['sentence'][0], word_vec_dict)


word_vector_size = 50;
time_steps = 100;
num_classes = 2
batch_size = 1000;
n_iterations = 10;
hidden_layer_size = 50

training_data = getData('train.csv')
training_rows  = map(lambda row: transform(row), training_data)
training_data = map(lambda row: row[1], training_rows)
training_labels = map(lambda row: row[0], training_rows)
#test_data = map(lambda row: transform(row), getData('test.csv'))
test_data = getData('test.csv')
test_rows  = map(lambda row: transform(row), test_data)
test_data = map(lambda row: row[1], test_rows)
test_labels = map(lambda row: row[0], test_rows)
print training_data[0]

[['0.68491' '0.32385' '-0.11592' ..., '0.17874' '-0.1693' '0.062375']
 ['0.96193' '0.012516' '0.21733' ..., '0.14032' '-0.38468' '-0.38712']
 ['0.6008' '0.18044' '0.078339' ..., '-0.016404' '-0.65372' '-0.38255']
 ..., 
 ['0.88387' '-0.14199' '0.13566' ..., '0.52711' '-0.20148' '0.0095952']
 ['-0.0010919' '0.33324' '0.35743' ..., '-0.45697' '-0.048969' '1.1316']
 ['-0.55114' '-0.16296' '-0.95494' ..., '-1.0346' '-0.25143' '1.4836']]


In [2]:

#Setting up the input and labels placeholders
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,
                                            word_vector_size])
y = tf.placeholder(tf.int32, shape=[None, num_classes])
y_one_hot = tf.one_hot( y , num_classes )

# TensorFlow built-in functions
# Creating the RNN cell and creating the outputs
with tf.variable_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size,
                                             forget_bias=1.0)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell,_inputs, dtype=tf.float32)

weights = {
        'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size,
                                                         num_classes],
                                                         mean=0,stddev=.01))
}


# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states[1],
                         weights["linear_layer"])
softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,
                                                  labels = y)                         
cross_entropy = tf.reduce_mean(softmax)

train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1),
                              tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100






In [ ]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

label_batch = np.array(training_labels[100 : 200])
l = np.array(dense_to_one_hot(label_batch,2)).reshape(len(label_batch), num_classes)

print l.shape

(100, 2)


In [ ]:
batch_size = 100;
n_iterations = 20;

#Initialize session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #Initialize test data
    #test_data = mnist.test.images[:batch_size].reshape((-1,time_steps, word_vector_size))
    #test_label = mnist.test.labels[:batch_size]

    for epoch in range(n_iterations):
        print "Epoch:", epoch
        for j in range(len(training_data)/batch_size):
            #print "j:", j
            if(j == 100):
                break
            startIndex = j*batch_size
            endIndex = startIndex + batch_size
            batch_x = np.array(training_data[startIndex : endIndex]).reshape((-1,time_steps, word_vector_size))
            print len(training_labels[startIndex : endIndex])
            batch_y = dense_to_one_hot(np.array(training_labels[startIndex : endIndex]),num_classes)
            print "X:", len(batch_x)
            print "Y:", len(batch_y)
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_x = batch_x.reshape((batch_size, time_steps, word_vector_size))
            sess.run(train_step, feed_dict={_inputs: batch_x,
                                            y: batch_y})
            if epoch % 1 == 0:
                acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
                loss = sess.run(cross_entropy, feed_dict={_inputs: batch_x, y: batch_y})
                print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                       "{:.6f}".format(loss) + ", Training Accuracy= " + \
                       "{:.5f}".format(acc))


    print ("Testing Accuracy:",
        sess.run(accuracy, feed_dict={_inputs: test_data, y: dense_to_one_hot(np.array(test_labels), num_classes)}))

Epoch: 0
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693461, Training Accuracy= 48.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692917, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692556, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692934, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692942, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693424, Training Accuracy= 45.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693386, Training Accuracy= 49.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692397, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693163, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692959, Training Accuracy= 56.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693013, Training Accuracy= 48.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692739, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minib

Iter 1, Minibatch Loss= 0.691238, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.691028, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.692887, Training Accuracy= 44.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.691343, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.693423, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.689382, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.692336, Training Accuracy= 50.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690685, Training Accuracy= 62.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690483, Training Accuracy= 58.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690878, Training Accuracy= 58.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690982, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.689376, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.689219, Traini

Iter 2, Minibatch Loss= 0.665891, Training Accuracy= 56.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.694913, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.612035, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.661540, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.613033, Training Accuracy= 69.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.571754, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.630923, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.621718, Training Accuracy= 71.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.603904, Training Accuracy= 69.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.611084, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.626574, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.622319, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.670457, Traini

Iter 3, Minibatch Loss= 0.617629, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.560818, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.533543, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.609249, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.583608, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.555367, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.515433, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.550708, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.568534, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.602704, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.604237, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.625899, Training Accuracy= 65.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.578739, Traini

Iter 4, Minibatch Loss= 0.579937, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.550144, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.634687, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.514708, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.528437, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.602011, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.590567, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.575224, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.654321, Training Accuracy= 64.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.613482, Training Accuracy= 64.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.533724, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.523096, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.635006, Traini

Iter 5, Minibatch Loss= 0.501177, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.519920, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.576000, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.578335, Training Accuracy= 69.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.577216, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.600135, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.540037, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.593491, Training Accuracy= 71.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.485833, Training Accuracy= 80.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.562176, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.575095, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.472982, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 5, Minibatch Loss= 0.581762, Traini